### Introduction
In python, there are many ways to read and write csv files. Here I explore 3 ways:<br /> 

(1) Pandas -- read_csv(), dataframe.to_csv()<br />
(2) csv library -- reader() and writer()<br />
(3) csv library -- DictReader() and DictWriter()<br />



In [5]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
from IPython.display import display

### Explore dataset

Explore the dataset I use for this presentation.

In [6]:
file_in = './data/201402_trip_data.csv'
trip_info = pd.read_csv(file_in)
display(trip_info.shape)
display(trip_info.head())
display(trip_info.tail())

(144015, 11)

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscription Type,Zip Code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscription Type,Zip Code
144010,198771,385,2/28/2014 22:15,Powell Street BART,39,2/28/2014 22:22,South Van Ness at Market,66,483,Subscriber,94404
144011,198772,145,2/28/2014 22:38,Commercial at Montgomery,45,2/28/2014 22:40,Davis at Jackson,42,425,Subscriber,94111
144012,198773,677,2/28/2014 22:45,Embarcadero at Sansome,60,2/28/2014 22:56,Market at 4th,76,438,Subscriber,94102
144013,198774,64128,2/28/2014 23:01,Civic Center BART (7th at Market),72,3/1/2014 16:50,Harry Bridges Plaza (Ferry Building),50,414,Customer,94124
144014,198775,570,2/28/2014 23:20,2nd at South Park,64,2/28/2014 23:30,Townsend at 7th,65,577,Subscriber,94107


### Method 1 - Using pandas library


In [55]:
##  Readin first a few lines, use Pandas

datafile_in = './data/201402_trip_data.csv'

file_in = pd.read_csv(datafile_in)

for i in range (len(file_in)):
    data = file_in.loc[i,'Start Date']
    if data[:9]== '10/1/2013':
        index = i 
        break
        
#i = 0
#while True:
#    data = file_in.loc[i,'Start Date']
#    if data[:9]== '10/1/2013':
#        index = i 
#        break
#    i+=1
        
file_out_df = file_in[:index]
#display(file_in.iloc[:,:3])
#print (file_out_df.loc[0])

file_out_df.to_csv('./data/201309_trip_data1.csv', index = False)   
    

In [36]:
# Display the first few rows of the station data file.
station_info = './data/201402_station_data.csv'
station_info = pd.read_csv(station_info)
print (len(station_info.index))
display(station_info.tail())
display(station_info.head())

69


,station_id,name,lat,long,dockcount,landmark,installation
64,76,Market at 4th,37.786305,-122.404966,19,San Francisco,8/25/2013
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,San Jose Government Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


,station_id,name,lat,long,dockcount,landmark,installation
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [37]:
def creat_station_mapping(station_data):
    # Mapping stationID to city
    id_to_city = {}
    
    station = pd.read_csv(station_data)
    for i in range(len(station)):
        id_to_city[station.iloc[i,0]] = station.iloc[i,5]

    return id_to_city

### Method 2 - csv.reader() and csv.writer()

In [32]:
## Generalize as a function
def extract_firstfewline_csv1 (datafile_in, datafile_out, criterion):
    with open (datafile_in, 'r') as file_in, open (datafile_out, 'w') as file_out:
        reader = csv.reader(file_in)
        writer = csv.writer(file_out, lineterminator ='\n')
        
        for item in reader:
            if item[2][:9] == criterion:
                break
                
            writer.writerow(item)
    

extract_firstfewline_csv1 ('./data/201402_trip_data.csv', './data/201309_trip_data2.csv', '10/1/2013') 

### Method 3 - csv.DictReader() and csv.DictWriter()

In [35]:
## Readin first a few lines, use csv.DictReader and csv.DictWriter, generalize a function

def extract_firstfewline_csv2 (datafile_in, datafile_out, header, criterion):
    with open (datafile_in, 'r') as f_in, open (datafile_out, 'w') as f_out:
        reader = csv.DictReader(f_in)
        writer = csv.DictWriter(f_out, fieldnames = header, lineterminator = '\n')
        writer.writeheader()
    
        for item in reader:
            if item ['Start Date'][:9] == criterion:
                break            
            writer.writerow(item)
        
header = ['Trip ID','Duration','Start Date','Start Station','Start Terminal','End Date','End Station','End Terminal','Bike #',
              'Subscription Type','Zip Code']
extract_firstfewline_csv2 ('./data/201402_trip_data.csv', './data/201309_trip_data3.csv', header, '10/1/2013')  

In [36]:
datafile_station1 = './data/201402_station_data.csv'
datafile_station2 = './data/201408_station_data.csv'
datafile_station3 = './data/201508_station_data.csv'

datafile = [datafile_station1,datafile_station2,datafile_station3]
ID_city_mapping = {}

for files in datafile:
    with open (files, 'r') as station:
        reader = csv.DictReader (station)
        for row in reader:
            ID_city_mapping[row['station_id']] = row['landmark']
   
            

In [37]:
trip1 = './data/201402_trip_data.csv'
trip2 = './data/201408_trip_data.csv'
trip3 = './data/201508_trip_data.csv'

trips = [trip1, trip2, trip3]

header = ['duration','start_date','start_year','start_month','start_hour','weekday','start_city','end_city','subscription_type']

with open ('./data/total_trip_summary.csv', 'w') as summary:
    writer = csv.DictWriter (summary, fieldnames = header, lineterminator= '\n')
    writer.writeheader()
    for trip in trips:
        with open (trip1, 'r') as trip_in: 
            reader = csv.DictReader (trip_in)
            
            for item in reader:
                row ={}
                trip_date = datetime.strptime(item['Start Date'], '%m/%d/%Y %H:%M')
                row ['duration'] = int(item['Duration'])/60.0
                row ['start_date'] = trip_date.strftime('%Y-%m-%d')
                row ['start_month'] = trip_date.strftime('%m')
                row ['start_hour'] = trip_date.strftime('%H')
                row ['weekday'] = trip_date.strftime('%H')
                row ['start_city'] = ID_city_mapping[item['Start Terminal']]
                row ['end_city'] = ID_city_mapping[item['End Terminal']]

                if 'Subscription Type' in item:
                    row ['subscription_type'] = item['Subscription Type']
                else:
                    row ['subscription_type'] = item['Subscriber Type']

                writer.writerow(row)
